In [1]:
# Importing necessary dependencies
import numpy as np
import math
import random
import copy

In [2]:
# Defining variables
n = 1000000   # n defines the number of photons generated
p = 1       # Percentage of bits compared on the classical channel

In [9]:
# Function to generate a binary sequence of size n
def bin_seq(n):
    arr = 2 * np.random.randint(0, 2, n) - 1  
    return arr

In [4]:
class User:
    '''User class: Defines the attributes and methods of a user sending information on a channel'''
    
    def __init__(self, n):
        '''Initialize a User object with specified attributes'''
        self.n = n
        self.basis = bin_seq(n)          # Basis: Defines the basis chosen by a user for producing polarized photons
        self.key = None                  # Key: Stores the initial binary sequence to be sent on the quantum channel
        
    def generate_bits(self):
        '''Generate random bits and store them in the key attribute'''
        self.key = bin_seq(self.n)
    
    def polarization(self):
        '''Generate polarized photons based on the key and basis'''
        photons = copy.deepcopy(self.key)
        for i in range(len(self.basis)):
            config = self.basis[i]
            if config == 1:
                photons[i] += config  
            else:
                photons[i] *= config 
        return photons
    
    def receive_bits(self, photons):
        '''Receive polarized photons and adjust based on the receiving end basis (reverse operations)'''
        for i in range(len(self.basis)):
            config = self.basis[i]
            if config == 1:
                photons[i] -= config
            else:
                photons[i] /= config
                
            if photons[i] % 2 == 0:
                photons[i] = (2 * np.random.randint(0, 2)) - 1  
                
        self.key = photons

In [5]:
class ClassicalChannel:
    '''Classical Channel class definition'''
    
    def __init__(self):
        '''Initialize the ClassicalChannel object'''
        return
    
    @staticmethod
    def compare_basis(basis_a, basis_b):
        '''Function to compare the basis of the two users'''
        indices = np.where(basis_a == basis_b)[0]
        return indices

    @staticmethod
    def compare_bits(photons_a, photons_b):
        '''Function to compare a given percentage of photons corresponding to the same basis'''
        if np.array_equal(photons_a, photons_b):
            print("Key Generation Successful")
            print(f'Your key of size {len(photons_a)} is {(photons_a+1)//2}')
        else:
            print("Eavesdropper detected")
            indices = np.where(photons_a != photons_b)[0]
            print(f'Interceptions detected at {len(indices)} places.')


In [6]:
# Successful key exchange case: No eavesdropper present on the channel 
alice = User(n)
bob = User(n)
#print("Alice's basis:", alice.basis,"\nBob's basis:", bob.basis)

alice.generate_bits()
#print("Alice's bit sequence", alice.key)
photons = alice.polarization()
#print("Alice's photons:", photons)
bob.receive_bits(copy.deepcopy(photons))
#print("Bob's bit sequence:", bob.key)

channel = ClassicalChannel
indices = channel.compare_basis(alice.basis, bob.basis)
ordered_indices = sorted(random.sample(list(indices), int(len(indices) * p)))
#print(ordered_indices)
channel.compare_bits(alice.key[ordered_indices], bob.key[ordered_indices])


Key Generation Successful
Your key of size 500201 is [1 0 0 ... 0 1 0]


In [7]:
# Unsuccessful secure key exchange case: Eavesdropper present on the channel
alice = User(n)
bob = User(n)
eve = User(n)
#print("Alice's basis:", alice.basis,"\nBob's basis:", bob.basis)

alice.generate_bits()
#print("Alice's bit sequence", alice.key)
photons = alice.polarization()
#print("Alice's photons:", photons)
eve.receive_bits(copy.deepcopy(photons))
#print(" Eve's bit sequence:", eve.key)
modified_photons = eve.polarization()
#print("Eve's photons:", modified_photons)
bob.receive_bits(copy.deepcopy(modified_photons))
#print("Bob's bit sequence:", bob.key)

channel = ClassicalChannel
indices = channel.compare_basis(alice.basis, bob.basis)
ordered_indices = sorted(random.sample(list(indices), int(len(indices) * p)))
#print(ordered_indices)
channel.compare_bits(alice.key[ordered_indices], bob.key[ordered_indices])

Eavesdropper detected
Interceptions detected at 124743 places.
